In this notebook we will be creating granular datasets around meme dates from out bulk datasets (twitter and FEC). The FEC data needs to be cleaned still, but not much needs to be done besides parsing the date. 

Our end df's will be grouped by date, daily by whatever our 'dependent' variable is, either polarity (tweets) or individual donation (FEC). We want a csv for each Meme, in total there will be 8 (4 for tweets and 4 for FEC). They should be 1 month before peak and 1 month after peak.

### Meme dates:
Bernie: 
- Bird On Podium: april 2, 2016
- Feel the bern: Feb 7-13 2016
##### need to do
- Feel the bern - April 10th now
- Barnie Sandlers - January 17th

Cruz:
- Zodiac Killer: feb 28 2016
- Zodiac Killer 2: May 1 2016

##### need to do
ted cruz look alike: April 30th


FEC:
- Grouped daily for sum donations.

Tweets:
- Grouped daily, average polarity


In [1]:
import pandas as pd
import csv
from textblob import TextBlob
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import numpy as np

## Tweets First

In [2]:
bernie = pd.concat([pd.read_csv(f) for f in ['bern2015-02-20.csv', 'bernie-2015-03-23T00:00:00+01:00.csv', 'bernie-2016-12-01T00:00:00+01:00.csv']])
cruz = pd.read_csv("cruz-2016-12-01T00:00:00+01:00.csv")

In [3]:
def format_date(date):
    year_month_day = date.split('T')[0].split('-')
    year = int(year_month_day[0])
    month = int(year_month_day[1])
    day = int(year_month_day[2])
    return datetime(year, month, day)

def get_sentiment(text:str):
    return TextBlob(text).sentiment.polarity

def clean_data(df: pd.DataFrame):
    df = df.drop_duplicates(subset='id', keep='first')
    
    # format to datetime
    df['created_at'] = df['created_at'].apply(format_date)
    df['polarity'] = df['text'].apply(get_sentiment)
        
    dfg = df.groupby('created_at').mean('polarity')
    
    
    return df, dfg

In [4]:
cruz, cruz_grouped = clean_data(cruz)
bernie, bernie_grouped = clean_data(bernie)

/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/1608257482.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = df['created_at'].apply(format_date)
/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/1608257482.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polarity'] = df['text'].apply(get_sentiment)
/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/1608257482.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [5]:
bernie_grouped = bernie_grouped.reset_index()
cruz_grouped = cruz_grouped.reset_index()

In [40]:
def is_meme(date, meme_date):
    if date >= meme_date:
        return 1
    return 0

def create_meme_df(df: pd.DataFrame, meme_peak_date: datetime, date_column: str = 'created_at'):
    begin = meme_peak_date - relativedelta(months=1)
    end = meme_peak_date + relativedelta(months=1)
    
    meme_df = df[(df[date_column] >= begin) & (df[date_column]<=end)]
    
    # create variable that is 1 after meme peak date
    meme_df['after_meme_peak'] = df[date_column].apply(lambda x: is_meme(x,meme_peak_date))
    
    return meme_df

In [36]:
# Bernie Sanders Meme dates
bird_on_podium_date = datetime(2016,4,2)
feel_bern_date = datetime(2016,4,10)
barnie_sandlers = datetime(2016,1,17)

# Ted Cruz meme dates
zodiac_1_date = datetime(2016, 2,28)
zodiac_2_date = datetime(2016,5,1)
look_alike=datetime(2016,4,30)

In [8]:
# bernie's meme dfs
b_o_p_df = create_meme_df(bernie_grouped, bird_on_podium_date)
feel_the_bern_df = create_meme_df(bernie_grouped, feel_bern_date)
barnie_sandlers_df = create_meme_df(bernie_grouped, barnie_sandlers)
# cruz's meme dfs
zodiac_1_df = create_meme_df(cruz_grouped, zodiac_1_date)
zodiac_2_df = create_meme_df(cruz_grouped, zodiac_2_date)
look_alike_df = create_meme_df(cruz_grouped, look_alike)


/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/2637655744.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meme_df['after_meme_peak'] = df[date_column].apply(lambda x: is_meme(x,meme_peak_date))
/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/2637655744.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meme_df['after_meme_peak'] = df[date_column].apply(lambda x: is_meme(x,meme_peak_date))
/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/2637655744.py

In [10]:
b_o_p_df.to_csv("data/twitter_bird_on_podium.csv")
feel_the_bern_df.to_csv("data/twitter_feel_the_bern.csv")
barnie_sandlers_df.to_csv("data/twitter_barnie_sandlers.csv")
zodiac_1_df.to_csv("data/twitter_zodaic_killer_1.csv")
zodiac_2_df.to_csv("data/twitter_zodiac_killer_2.csv")
look_alike_df.to_csv("data/twitter_look_alike.csv")

### FEC data Now

In [14]:
bernie_fec = pd.read_csv("bernie-2015-2016-FEC.csv")
cruz_fec = pd.read_csv("cruz-2015-2016-FEC.csv")

/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/4000196023.py:1: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  bernie_fec = pd.read_csv("bernie-2015-2016-FEC.csv")
/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/4000196023.py:2: DtypeWarning: Columns (5,10,15,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  cruz_fec = pd.read_csv("cruz-2015-2016-FEC.csv")


In [15]:
def parseDate(date: float):
    """parse date from the float number in original dataset"""
    date_num_str = str(int(date))
    year = int(date_num_str[-4:])
    day = int(date_num_str[-6:-4])
    month = int(date_num_str[-8:-6])
    
    return datetime(year, month, day)

def clean_fec_data(df: pd.DataFrame):
    df['TRANSACTION_DT']=df['TRANSACTION_DT'].apply(parseDate)
    dg = df[['TRANSACTION_DT', 'TRANSACTION_AMT']]

    dg = dg.groupby('TRANSACTION_DT').sum('TRANSACTION_AMT')
    dg = dg.reset_index()
    
    return dg

In [16]:
bernie_fec_grouped = clean_fec_data(bernie_fec)
cruz_fec_grouped = clean_fec_data(cruz_fec)


In [27]:
date_column = "TRANSACTION_DT"
# bernie's meme dfs
b_o_p__fec_df = create_meme_df(bernie_fec_grouped, bird_on_podium_date, date_column)
feel_the_bern_fec_df = create_meme_df(bernie_fec_grouped, feel_bern_date, date_column)
barnie_sandlers_fec_df = create_meme_df(bernie_fec_grouped, barnie_sandlers, date_column)
# cruz's meme dfs
zodiac_1_fec_df = create_meme_df(cruz_fec_grouped, zodiac_1_date, date_column)
zodiac_2_fec_df = create_meme_df(cruz_fec_grouped, zodiac_2_date, date_column)
look_alike_fec_df = create_meme_df(cruz_fec_grouped, look_alike, date_column)

/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/2219568398.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meme_df['after_meme_peak'] = df[date_column].apply(lambda x: is_meme(x,meme_peak_date))


In [28]:
b_o_p__fec_df.to_csv("data/fec_bird_on_podium.csv")
feel_the_bern_fec_df.to_csv("data/fec_feel_the_bern.csv")
barnie_sandlers_fec_df.to_csv("data/fec_barnie_sandlers.csv")
zodiac_1_fec_df.to_csv("data/fec_zodaic_killer_1.csv")
zodiac_2_fec_df.to_csv("data/fec_zodiac_killer_2.csv")
look_alike_fec_df.to_csv("data/fec_look_alike.csv")

# Tweet Volume extra credit

Now we will make some meme datasets for tweet volume

In [32]:
bernie_volume = pd.read_csv("data/bernie-tweet-volume.csv")
cruz_volume = pd.read_csv("data/cruz-tweet-volume.csv")

In [33]:
bernie_volume['start'] = bernie_volume['start'].apply(format_date)
cruz_volume['start'] = cruz_volume['start'].apply(format_date)

In [34]:
bernie_volume

,end,start,tweet_count
0,2016-10-31T00:00:00.000Z,2016-10-30,7764
1,2016-11-01T00:00:00.000Z,2016-10-31,9084
2,2016-11-02T00:00:00.000Z,2016-11-01,14352
3,2016-11-03T00:00:00.000Z,2016-11-02,15841
4,2016-11-04T00:00:00.000Z,2016-11-03,18762
...,...,...,...
695,2015-01-14T00:00:00.000Z,2015-01-13,1477
696,2015-01-15T00:00:00.000Z,2015-01-14,2122
697,2015-01-16T00:00:00.000Z,2015-01-15,1629
698,2015-01-17T00:00:00.000Z,2015-01-16,1286


In [41]:
date_column = "start"

b_o_p_volume_df = create_meme_df(bernie_volume, bird_on_podium_date, date_column)
feel_the_bern_volume_df = create_meme_df(bernie_volume, feel_bern_date, date_column)
barnie_sandlers_volume_df = create_meme_df(bernie_volume, barnie_sandlers, date_column)
# cruz's meme dfs
zodiac_1_volume_df = create_meme_df(cruz_volume, zodiac_1_date, date_column)
zodiac_2_volume_df = create_meme_df(cruz_volume, zodiac_2_date, date_column)
look_alike_volume_df = create_meme_df(cruz_volume, look_alike, date_column)

/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/2637655744.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meme_df['after_meme_peak'] = df[date_column].apply(lambda x: is_meme(x,meme_peak_date))
/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/2637655744.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meme_df['after_meme_peak'] = df[date_column].apply(lambda x: is_meme(x,meme_peak_date))
/var/folders/kx/ff4_40hx5q993v5z30g97l3w0000gn/T/ipykernel_96413/2637655744.py

In [47]:
b_o_p_volume_df.to_csv("data/volume_bird_on_podium.csv")
feel_the_bern_volume_df.to_csv("data/volume_feel_the_bern.csv") 
barnie_sandlers_volume_df.to_csv("data/volume_barnie_sandlers.csv") 
# cruz's meme dfs
zodiac_1_volume_df.to_csv("data/volume_zodiac_1.csv") 
zodiac_2_volume_df.to_csv("data/volume_zodiac_2.csv")
look_alike_volume_df.to_csv("data/volume_look_alike.csv")